In [ ]:
import pandas as pd
import spacy
from spacy import displacy
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
#import flair
#from flair.nn import Classifier
#from flair.data import Sentence, Corpus
#from flair.models import SequenceTagger
#from flair.datasets import CONLL_03, CONLL_03_GERMAN
#from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings
#https://huggingface.co/flair/ner-multi
#https://spacy.io/usage/training

ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [63]:
tagged = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/eBay_Competition/Train_Tagged_Titles.tsv', sep='\t',quoting=3)
tagged

,Record Number,Title,Token,Tag
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,NaN
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ
...,...,...,...,...
55178,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Sportschuhe,Produktart
55179,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Wanderschuh,NaN
55180,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Big,No Tag
55181,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Size,No Tag


In [ ]:
#We actually should not use forward fill because then we can't differentiate if the groupings are the same or not
#If the field is empty that indicates the token in that row belongs to the same semantic entity as the token before it, in other words, the title has a multi-token entity. In this case the tag of the previous row would apply to the current token and the tokens would need to be combined with a single whitespace to obtain the corresponding aspect value. If two (or more) consecutive rows have the same non-missing entry present in the Tag field, it means they have the same tag, but are different entities, and should not be combined.
#Somethings we did not talk about:
#Repeated token: No tag
#If an aspect value consists of two or more tokens then the tokens should be concatenated with spaces in between to form the aspect value, even if that space only happens to occur as a consequence of the tokenization.
#Supreme By any Means-> modell even though they are separately tagged
#In general, tokens that are literal aspect names such as the words "Muster" (pattern), "Farbe" (color) or "Größe" (size) should not be tagged. In rare cases, however, they have a semantic function (i.e. are part of the phrase being tagged) and should be tagged. In "Farbe Weiß" (color white) only "Weiß" should be tagged with the aspect name "Farbe" (color), but the token "Farbe" should not be tagged.

In [ ]:
tagged[tagged['Record Number']==1]

,Record Number,Title,Token,Tag
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,Produktlinie
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ
5,1,Supreme Nike SB Dunk High By any Means Red US1...,By,Modell
6,1,Supreme Nike SB Dunk High By any Means Red US1...,any,Modell
7,1,Supreme Nike SB Dunk High By any Means Red US1...,Means,Modell
8,1,Supreme Nike SB Dunk High By any Means Red US1...,Red,Farbe
9,1,Supreme Nike SB Dunk High By any Means Red US1...,US10,US-Schuhgröße


In [ ]:
tagged.iloc[1]['Title']

'Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force'

In [ ]:
nlp = spacy.load("en_core_web_sm") #Need to !python -m spacy download de_core_news_sm (These titles are in English and German**)
doc=nlp('Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force')
for ent in doc.ents:
  print(ent.text, "|", ent.label_, "|", spacy.explain(ent.label_))

NameError: ignored

In [ ]:
displacy.render(doc,style='ent', jupyter=True)
#Does not recognize Supreme now. We need to actually load multilanguage model that supports both German and English

In [ ]:
nlp.pipe_labels['ner']
#what spacy model supports

['LOC', 'MISC', 'ORG', 'PER']

In [ ]:
#Span in SpaCy: subString
type(doc[2:4])

spacy.tokens.span.Span

In [ ]:
doc[14:16]

Air Force

In [ ]:
from spacy.tokens import Span
#Can do custom labelization via SpaCy
s1=Span(doc,2,4,label="Produktlinie")
s2=Span(doc,14,16,label="Produktlinie")
doc.set_ents([s1,s2],default='unmodified')

In [ ]:
for ent in doc.ents:
  print(ent.text, "|", ent.label_)

Nike | PER
SB Dunk | Produktlinie
High By any | PER
Air Force | Produktlinie


In [ ]:
#Approach: Simple Lookup(operator manually adds entities in the database)
#Rule based NER: If it's a noun, then set it to this NER (Spacy-EntityRuler)
#-> List of entities and patterns
#Conditional Random Fields(CRF) and BERT
#Bert: they incorporate positional encodings to give the model information about the position of words in a sequence
#-> attention mechnaism: allows model to weigh the importance of different words in a sentence relative to given word
#-> feed-forward neural network
#-> layer normalization
#LSTM: word order built-in
#George Washington vs Washington


In [ ]:
#!pip install flair
tagger=Classifier.load('ner')
sentence=Sentence('Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force')
tagger.predict(sentence)

2023-10-21 23:54:03,153 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Sentence[16]: "Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force" → ["Air Force"/ORG]


In [ ]:
# load tagger
taggermulti = SequenceTagger.load("flair/ner-multi")

# make example sentence in any of the four languages
sentence = Sentence("George Washington ging nach Washington")

# predict NER tags
taggermulti.predict(sentence)

2023-10-22 01:17:58,774 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, B-PER, E-PER, S-LOC, B-MISC, I-MISC, E-MISC, S-PER, B-ORG, E-ORG, S-ORG, I-ORG, B-LOC, E-LOC, S-MISC, I-PER, I-LOC, <START>, <STOP>


In [ ]:
print(sentence)
# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)

Sentence[5]: "George Washington ging nach Washington" → ["George Washington"/PER, "Washington"/LOC]
The following NER tags are found:
Span[0:2]: "George Washington" → PER (0.9851)
Span[4:5]: "Washington" → LOC (0.991)


In [ ]:
sentence1 = Sentence('Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force')
taggermulti.predict(sentence1)

In [ ]:
for label in sentence1.get_labels():
    print(label)

Span[0:2]: "Supreme Nike" → ORG (0.7393)
Span[2:4]: "SB Dunk" → PER (0.607)
Span[7:9]: "Means Red" → ORG (0.8212)
Span[14:16]: "Air Force" → ORG (0.9753)


In [ ]:
print(sentence1)
# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence1.get_spans('ner'):
    print(entity)

Sentence[16]: "Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force" → ["Supreme Nike"/ORG, "SB Dunk"/PER, "Means Red"/ORG, "Air Force"/ORG]
The following NER tags are found:
Span[0:2]: "Supreme Nike" → ORG (0.7393)
Span[2:4]: "SB Dunk" → PER (0.607)
Span[7:9]: "Means Red" → ORG (0.8212)
Span[14:16]: "Air Force" → ORG (0.9753)


In [ ]:
!python -m spacy download xx_ent_wiki_sm

2023-10-22 06:03:25.181675: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 51.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')


In [64]:
tagged.head(10)

,Record Number,Title,Token,Tag
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,NaN
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ
5,1,Supreme Nike SB Dunk High By any Means Red US1...,By,Modell
6,1,Supreme Nike SB Dunk High By any Means Red US1...,any,NaN
7,1,Supreme Nike SB Dunk High By any Means Red US1...,Means,NaN
8,1,Supreme Nike SB Dunk High By any Means Red US1...,Red,Farbe
9,1,Supreme Nike SB Dunk High By any Means Red US1...,US10,US-Schuhgröße


In [70]:
pd.isna(tagged.iloc[3]['Tag'])

True

**Preparing Training Data for Spacy**:


In [88]:
#Preparing Spacy Tagged Training Data
#Resource link: https://www.machinelearningplus.com/nlp/training-custom-ner-model-in-spacy/
def prep_training_data(df):
  index=0
  start_index=0
  train_data=[]
  while index<len(df):
    row=df.iloc[index]
    record_number = row['Record Number']
    title = row['Title']
    token=row['Token']
    tag = row['Tag']
    # Find the starting index of the token in the title starting from the current start_index
    start_index = title.find(token, start_index)
    end_index=start_index+len(token)
    while index+1<len(df) and df.iloc[index+1]['Title']==title and (df.iloc[index + 1]['Tag'] == 'N/A' or pd.isna(df.iloc[index+1]['Tag'])):
      index += 1
      next_row = df.iloc[index]
      next_token = next_row['Token']
      end_index = next_row['Title'].find(next_token, end_index) +len(next_token)
    train_data.append((df['Title'].iloc[index], {"entities":[(start_index,end_index,tag)]}))
    if index+1<len(df) and title!=df['Title'].iloc[index+1]:
      start_index=0
    index+=1
  return train_data
spacy_data=prep_training_data(tagged)

In [90]:
tagged['Title'].iloc[0][13:20]

'SB Dunk'

In [91]:
tagged['Title'].iloc[0][26:38]

'By any Means'

In [89]:
spacy_data

[('Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force',
  {'entities': [(0, 7, 'Modell')]}),
 ('Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force',
  {'entities': [(8, 12, 'Marke')]}),
 ('Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force',
  {'entities': [(13, 20, 'Produktlinie')]}),
 ('Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force',
  {'entities': [(21, 25, 'Schuhschaft-Typ')]}),
 ('Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force',
  {'entities': [(26, 38, 'Modell')]}),
 ('Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force',
  {'entities': [(39, 42, 'Farbe')]}),
 ('Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force',
  {'entities': [(43, 47, 'US-Schuhgröße')]}),
 ('Supreme Nike SB Dunk High By any Means Red US10 EU44 Supreme Box Logo Air Force',
  {'entities': [(48, 52, 'EU-S

In [99]:
spacy_data

[('Puma Rebound Street v2 L Herren Schwarz Schuhe Sneaker 363716-01 NEU',
  {'entities': [(65, 68, 'No Tag')]}),
 ('Nike Air Max LTD 3 Gr . wählbar 687977 020 631 Turnschuhe Sneakers',
  {'entities': [(24, 31, 'No Tag')]}),
 ('New Balance WH 574 Mid Damen Sneaker Winterschuhe Leder Schuhe 1 Paar Neu rosa',
  {'entities': [(0, 11, 'Marke')]}),
 ('Adidas Tubular Doom Sock core black Sneaker schwarz BY3563',
  {'entities': [(7, 24, 'Modell')]}),
 ('Nike Dunk Low Easter 2022 ( W ) verschiedene Größen / NEU | OVP | RECHNUNG',
  {'entities': [(28, 29, 'Abteilung')]}),
 ('Reebok Instapump Fury Damen Sneaker Freizeit Sportschuhe Trainingschuhe BD1506',
  {'entities': [(28, 35, 'Stil')]}),
 ('2021D * io * r Sneaker High Freizeit Schuhe Mode Turnschuhe Herren Damen Laufschuhe',
  {'entities': [(6, 7, 'No Tag')]}),
 ('NEU Hummel Off-Field Schuhe Low Sneaker Unisex Retro Leder Sportschuhe SALES',
  {'entities': [(59, 70, 'Produktart')]}),
 ('Damen Stiefeletten Boots Gefüttert Plateau Sneaker Winte

In [102]:
import random

# Assuming you have a list of tuples 'spacy_data' as described in your question

# Set the random seed for reproducibility
random.seed(42)

# Define the ratio for the split (e.g., 80% train and 20% test)
train_ratio = 0.8

# Calculate the number of samples for each split
num_samples = len(spacy_data)
num_train_samples = int(train_ratio * num_samples)
num_test_samples = num_samples - num_train_samples

# Shuffle the data
random.shuffle(spacy_data)

# Split the data into train and test
train_data = spacy_data[:num_train_samples]
test_data = spacy_data[num_train_samples:]

In [101]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [105]:
# Import requirements
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training.example import Example

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(train_data)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_data, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        for i in range(len(texts)):
          doc = nlp.make_doc(texts[i])
          example.append(Example.from_dict(doc, annotations[i]))
        nlp.update(example, drop=0.5, losses=losses)
        print("Losses", losses)

Losses {'ner': 7.504625180998119}
Losses {'ner': 15.076683943654643}
Losses {'ner': 22.96990634533904}
Losses {'ner': 30.246683315444898}
Losses {'ner': 37.91195944133324}
Losses {'ner': 45.64751841321231}
Losses {'ner': 53.419669329259705}
Losses {'ner': 61.34156799276934}
Losses {'ner': 68.92600471126025}
Losses {'ner': 76.1969615216276}
Losses {'ner': 83.59838665048846}
Losses {'ner': 90.73090811777183}
Losses {'ner': 98.3400560222126}
Losses {'ner': 106.83610029351803}
Losses {'ner': 115.08972334543614}
Losses {'ner': 122.06698482258616}
Losses {'ner': 129.0140775933305}
Losses {'ner': 136.94248451162093}
Losses {'ner': 144.3682602277022}
Losses {'ner': 152.27724746717104}
Losses {'ner': 159.98315110788644}
Losses {'ner': 167.49001260460318}
Losses {'ner': 175.45328299354696}
Losses {'ner': 183.31990915429014}
Losses {'ner': 191.73390741746516}
Losses {'ner': 199.8622086020896}
Losses {'ner': 207.8113687868008}
Losses {'ner': 215.31090451039017}
Losses {'ner': 222.64704717827954}
L

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Knack 040 Modische Sneakers Herren Schuhe Turnschu..." with entities "[(53, 58, 'Anlass')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses {'ner': 457.8804605806981}
Losses {'ner': 465.65456630650806}
Losses {'ner': 472.71500392887174}
Losses {'ner': 479.2752762723285}
Losses {'ner': 487.1700872976835}
Losses {'ner': 494.9892186590041}
Losses {'ner': 502.9219008076507}
Losses {'ner': 509.958200807454}
Losses {'ner': 517.8740824118031}
Losses {'ner': 525.5779280448878}
Losses {'ner': 532.6455728810361}
Losses {'ner': 540.4177701070985}
Losses {'ner': 548.0064301349303}
Losses {'ner': 555.7720609576172}
Losses {'ner': 562.8822547859436}
Losses {'ner': 570.2001579041516}
Losses {'ner': 576.836450764762}
Losses {'ner': 582.9789639193023}
Losses {'ner': 591.2695941690745}
Losses {'ner': 598.8991376814356}
Losses {'ner': 606.23303488278}
Losses {'ner': 613.9643501942901}
Losses {'ner': 621.8694065169311}
Losses {'ner': 629.4398076359994}
Losses {'ner': 636.6470444191127}
Losses {'ner': 644.3064153937503}
Losses {'ner': 651.722539713699}
Losses {'ner': 659.4280468998822}
Losses {'ner': 667.4755374119031}
Losses {'ner': 67

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Nike Air Shake Ndestrukt / Pigalle AA4315 100 US_9..." with entities "[(73, 74, 'No Tag')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses {'ner': 3088.2754748552443}
Losses {'ner': 3097.245019199826}
Losses {'ner': 3106.803181995788}
Losses {'ner': 3115.917143286356}
Losses {'ner': 3125.5784498519834}
Losses {'ner': 3135.999246968747}
Losses {'ner': 3145.6398123631707}
Losses {'ner': 3155.219234666646}
Losses {'ner': 3164.4867495558583}
Losses {'ner': 3174.4980349540137}
Losses {'ner': 3184.359730032914}
Losses {'ner': 3194.2628502002344}
Losses {'ner': 3203.775859533286}
Losses {'ner': 3213.2650160261337}
Losses {'ner': 3222.622552005844}
Losses {'ner': 3231.964428110551}
Losses {'ner': 3241.4640841086302}
Losses {'ner': 3251.0658407643723}
Losses {'ner': 3260.4596737531765}
Losses {'ner': 3270.3774281681335}
Losses {'ner': 3279.373104215302}
Losses {'ner': 3288.991152706084}
Losses {'ner': 3298.640591872151}
Losses {'ner': 3307.416051173809}
Losses {'ner': 3316.363511390512}
Losses {'ner': 3326.0470658647764}
Losses {'ner': 3335.8780038054038}
Losses {'ner': 3345.281063128722}
Losses {'ner': 3355.6873821115523}


...

Losses {'ner': 59601.346332818386}
Losses {'ner': 59662.60087704903}
Losses {'ner': 59722.1508932967}
Losses {'ner': 59782.21136084723}
Losses {'ner': 59841.73199577036}
Losses {'ner': 59901.186904940405}
Losses {'ner': 59961.35277576913}
Losses {'ner': 60021.09342383576}
Losses {'ner': 60080.62524135021}
Losses {'ner': 60140.25072826847}
Losses {'ner': 60201.74204181641}
Losses {'ner': 60262.10602028057}
Losses {'ner': 60322.06270847999}
Losses {'ner': 60382.16772406566}
Losses {'ner': 60442.34414982394}
Losses {'ner': 60503.42527149155}
Losses {'ner': 60563.41798223634}
Losses {'ner': 60624.25078260488}
Losses {'ner': 60684.594629695726}
Losses {'ner': 60746.02114872903}
Losses {'ner': 60806.80300993275}
Losses {'ner': 60864.89306096163}
Losses {'ner': 60924.521068915616}
Losses {'ner': 60984.05517207215}
Losses {'ner': 61044.613007931745}
Losses {'ner': 61102.635205408194}
Losses {'ner': 61160.27292338885}
Losses {'ner': 61220.39441689289}
Losses {'ner': 61282.04518355614}
Losses {'

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Schuhe Skechers Arch Fit - Big Appeal 149057-BBK -..." with entities "[(44, 45, 'No Tag')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses {'ner': 64869.57444439699}
Losses {'ner': 64929.13848349788}
Losses {'ner': 64987.731355962445}
Losses {'ner': 65046.08765138277}
Losses {'ner': 65106.481297765495}
Losses {'ner': 65166.5392271882}
Losses {'ner': 65225.832470552275}
Losses {'ner': 65288.46425575039}
Losses {'ner': 65349.4177664893}
Losses {'ner': 65410.29120503713}
Losses {'ner': 65469.35940823958}
Losses {'ner': 65530.05528952412}
Losses {'ner': 65590.69999943717}
Losses {'ner': 65651.47421418547}
Losses {'ner': 65712.77714604369}
Losses {'ner': 65773.00714870574}
Losses {'ner': 65832.08576719309}
Losses {'ner': 65890.69307898692}
Losses {'ner': 65951.82711851838}
Losses {'ner': 66011.73270567748}
Losses {'ner': 66070.1740797045}
Losses {'ner': 66130.58216056501}
Losses {'ner': 66190.94675284153}
Losses {'ner': 66250.90926521103}
Losses {'ner': 66310.10355955473}
Losses {'ner': 66371.13787796706}
Losses {'ner': 66428.85308751077}
Losses {'ner': 66490.867148775}
Losses {'ner': 66550.87013395349}
Losses {'ner': 6

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "+ + + ORIGINAL ADIDAS STAN SMITH I WHITE I TOP I U..." with entities "[(37, 38, 'No Tag')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Losses {'ner': 48997.05165916687}
Losses {'ner': 49053.72840840189}
Losses {'ner': 49109.54811697459}
Losses {'ner': 49165.28374595946}
Losses {'ner': 49220.94111729735}
Losses {'ner': 49278.045858339734}
Losses {'ner': 49335.35978259101}
Losses {'ner': 49391.11050549733}
Losses {'ner': 49446.91030801048}
Losses {'ner': 49502.398607041876}
Losses {'ner': 49561.969592617075}
Losses {'ner': 49619.340896874615}
Losses {'ner': 49677.827952043}
Losses {'ner': 49736.9181634873}
Losses {'ner': 49794.03464835192}
Losses {'ner': 49850.703433842944}
Losses {'ner': 49909.610467125276}
Losses {'ner': 49967.97499852496}
Losses {'ner': 50025.76995517554}
Losses {'ner': 50084.05283108303}
Losses {'ner': 50144.73834549026}
Losses {'ner': 50205.06336883218}
Losses {'ner': 50264.207258194176}
Losses {'ner': 50323.69457586152}
Losses {'ner': 50381.92151600537}
Losses {'ner': 50441.95761594591}
Losses {'ner': 50500.23774480862}
Losses {'ner': 50560.040227

In [110]:
output_dir = Path('/content/')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to /content


In [113]:
doc = nlp("LACOSTE ESPARRE 319 3 CMA [ GR . 40 / / 42 ] HERREN LEDER SNEAKER BLAU NEU & OVP")
doc

LACOSTE ESPARRE 319 3 CMA [ GR . 40 / / 42 ] HERREN LEDER SNEAKER BLAU NEU & OVP

In [109]:
for i, j in test_data:
  print(i)

Streaming output truncated to the last 5000 lines.
Adidas X Tokio Solar Human Made Sneaker Turnschuhe weiß FZ0551 Gr . 37 - 47 NEU
Herren Schuhe Salomon Speedcross 3 Outdoorschuhe Laufschuhe Sportschuhe
K-SWISS Herren Schuhe schwarz Leder AVERY Sneaker Sportschuhe 39 - 46 NEU --
Lacoste Crosier Sail 7 SRM brown Schuhe / Sneaker BRW braun
adidas ZX 700 W Damen Sneaker Freizeitschuh schwarz ( S79795 ) Gr : Wählbar neu in K
adidas Originals Damen-Superstar Sneaker Schwarz Lackleder Turnschuhe Schuhe NEU
adidas Falcon W gr 39 40 42,5 43 B28128 Retro Style yung 1
Nike Air Jordan 1 Mid Mystic Navy Mint | Verschiedene Größen ✅
SKECHERS Sneaker Gr . 36 bordeaux memory foam Freizeit Schuhe Sportschuh neu
Nike Air Max 720-818 Frsh ( GS ) Gr . wählbar Neu & OVP CW4721 100
Puma Axelion Block Herren Schuhe Sneaker Turnschuhe 193148 ( Weiß 02 )
NEU SKECHERS Damen Sneakers Turnschuh Memory Foam D'LITES - REINVENTION Grau
adidas NMD R1 V2 H01485 Schwarz Black Sneaker Turnschuhe Herren Sport Originals


In [108]:
test_data

[('Mizuno Wave Stealth Herren Halbschuh Laufschuh Sportschuh Mehrfarbig Gr . 48,5',
  {'entities': [(58, 68, 'No Tag')]}),
 ('adidas Daily 3.0 Herren Sneaker Turnschuhe Sportschuhe H04569',
  {'entities': [(0, 6, 'Marke')]}),
 ('adidas Originals SL80 A SPZL , Sneaker Orange , Gr . 44 , F35877',
  {'entities': [(17, 28, 'Modell')]}),
 ('Onitsuka Tiger Mexico 66 Damen Leder Gelb Schwarz Turnschuhes EU Große 36 - 42',
  {'entities': [(76, 78, 'EU-Schuhgröße')]}),
 ('Baabuk Sky Wooler Sneaker Black Edition Merino Schurwolle Wolle Eco friendly',
  {'entities': [(64, 76, 'Besonderheiten')]}),
 ('Candice Cooper Sneaker , hightop , bunt , Leder & Stoff , Damen , Gr . 38',
  {'entities': [(15, 22, 'Stil')]}),
 ('adidas Originals ZX 500 RM Boost - Snake - BD7873 Herren Sneaker Turnschuhe NEU',
  {'entities': [(27, 32, 'Produktlinie')]}),
 ('ASICS GEL-LYTE V SANZE KNIT ( GRAU ) Gr . 41,5',
  {'entities': [(15, 22, 'Modell')]}),
 ('Neu 2021 AJ4 Lila-rot Freizeitsportschuhe Metallic ( CT8527-114 )'

In [ ]:
tagger = SequenceTagger.load("flair/upos-multi")

# make example sentence
sentence = Sentence("Ich liebe Berlin, as they say. ")

# predict POS tags
tagger.predict(sentence)

# print sentence
print(sentence)

# iterate over tokens and print the predicted POS label
print("The following POS tags are found:")
for token in sentence:
    print(token.get_label("upos"))


AttributeError: ignored

In [ ]:
from flair.data import Sentence
from flair.nn import Classifier
#Gives relationship
tagger = Classifier.load('ner-fast')
tagger.predict(sentence)

# check which named entities have been found in the sentence
entities = sentence.get_labels('ner')
for entity in entities:
    print(entity)

# 3. load relation extractor
extractor = Classifier.load('relations')

# predict relations
extractor.predict(sentence)

# check which relations have been found
relations = sentence.get_labels('relation')
for relation in relations:
    print(relation)

# Use the `get_labels()` method with parameter 'relation' to iterate over all relation predictions.
for label in sentence.get_labels('relation'):
    print(label)

2023-10-21 23:58:33,883 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Span[11:16]: "Supreme Box Logo Air Force" → ORG (0.7125)
2023-10-21 23:58:35,089 https://nlp.informatik.hu-berlin.de/resources/models/relations/relations-v11.pt not found in cache, downloading to /tmp/tmpx797xrbw


100%|██████████| 254M/254M [00:15<00:00, 17.1MB/s]

2023-10-21 23:58:51,328 copying /tmp/tmpx797xrbw to cache at /root/.flair/models/relations-v11.pt


2023-10-21 23:58:53,058 removing temp file /tmp/tmpx797xrbw


In [ ]:
#Pretrained model +1 layer

,Record Number,Title,Token,Tag
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,Produktlinie
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ
...,...,...,...,...
55178,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Sportschuhe,Produktart
55179,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Wanderschuh,Produktart
55180,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Big,No Tag
55181,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,Size,No Tag
